In [1]:
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

np.random.seed(2020)
random.seed(2020)

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import plot_confusion_matrix, accuracy_score
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [3]:
import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV

# Load data

In [4]:
pd.option_context('display.max_columns', None)

In [5]:
PATH = 'data/'

In [6]:
students_courses = pd.read_csv(PATH + 'train.csv').set_index('studentId')
students_courses.head()

,createdDate,updatedDate,isExpelled,expellingReason,courseCompleted,isTopPerformer,preferedMentorGithubId,readyFullTime,courseId,userId,...,hiredById,hiredByName,isFailed,startDate,endDate,repository,repositoryLastActivityDate,idUser,educationHistory,englishLevel
studentId,,,,,,,,,,,,,,,,,,,,,
5740,2019-09-06 17:30:17.681556,2019-12-03 16:24:58.705742,t,No activity in private repository,f,True,NaN,NaN,11,4467,...,NaN,NaN,f,2019-09-09 07:35:20.981+00,2019-12-04 09:17:02.998689+00,https://github.com/rolling-scopes-school/1b3f6...,NaN,4467,[],NaN
6956,2019-09-09 18:24:04.90096,2019-11-20 22:00:13.036321,t,No activity in private repository,f,True,NaN,NaN,11,5580,...,NaN,NaN,f,2019-09-09 07:35:20.981+00,2019-12-04 09:17:02.998689+00,https://github.com/rolling-scopes-school/76c0e...,NaN,5580,[],NaN
11616,2020-02-05 12:39:48.166948,2020-02-05 12:39:48.166948,f,NaN,f,False,NaN,NaN,16,9557,...,NaN,NaN,f,2020-02-05 12:39:48.164+00,NaN,NaN,NaN,9557,[],NaN
11773,2020-02-05 17:16:46.874484,2020-02-05 17:16:46.874484,f,NaN,f,False,NaN,NaN,16,9627,...,NaN,NaN,f,2020-02-05 17:16:46.872+00,NaN,NaN,NaN,9627,"[{""graduationYear"":null,""faculty"":null,""univer...",b1
6724,2019-09-09 17:06:56.967084,2020-01-19 16:55:12.069984,t,Failed Stage 2,f,True,NaN,NaN,11,5390,...,NaN,NaN,f,2019-09-09 07:35:20.981+00,1970-01-01 00:00:00+00,https://github.com/rolling-scopes-school/7fb23...,NaN,5390,[],NaN


In [7]:
students_courses['courseId'].isna().mean().round(4) * 100

0.0

In [8]:
courses = pd.read_csv(PATH + 'course.csv')
courses.head()

,courseId,createdDate,updatedDate,name,year,primarySkillId,primarySkillName,locationName,alias,completed,description,descriptionUrl,planned,courseStartDate,courseEndDate,fullName,registrationEndDate,inviteOnly
0,16,2020-01-27 06:15:21.813898,2020-06-02 09:49:08.084615,RSS React 2020,NaN,javascript,JavaScript,NaN,react-2020-Q1,t,Курс по React/Redux,NaN,f,2020-02-05 06:12:15.345+00,2020-03-18 06:12:18.391+00,RSS React 2020,2020-02-09 06:12:31.757+00,t
1,23,2020-02-25 09:28:08.842897,2020-07-28 06:07:15.142387,TEST COURSE,NaN,javascript,JavaScript,NaN,test-course,f,TEST COURSE,NaN,f,2020-02-25 09:28:00.105+00,2020-08-06 09:28:01.698+00,TEST COURSE,NaN,t
2,18,2020-02-11 08:40:16.050523,2020-09-10 18:30:58.028305,iOS 2020,NaN,ios-obj-c,iOS Objective-C,NaN,ios-2020,f,iOS 2020 course,NaN,f,2020-02-17 08:39:56.164+00,2020-06-01 08:40:00.109+00,Rolling Scopes iOS 2020,2020-03-08 09:33:55.611+00,f
3,9,2019-05-24 15:20:38.564615,2019-09-09 16:14:35.099569,RS 2018 Q1,2018.0,javascript,JavaScript,Minsk,rs-2018-q1,t,2018,NaN,f,2017-12-31 21:00:00.01+00,2018-06-30 21:00:00.01+00,Rolling Scopes School 2018 Q1,NaN,f
4,6,2019-05-22 11:56:03.343815,2019-09-09 16:14:58.34156,RS 2016 Q3,2016.0,javascript,JavaScript,Minsk,rs-2016-q3,t,RS 2016 Q3,NaN,f,2016-09-01 19:48:14.839+00,2016-12-31 19:48:14.839+00,Rolling Scopes School 2016 Q3,NaN,f


In [9]:
courses['courseId'].isna().mean().round(4) * 100

0.0

In [10]:
tasks = pd.read_csv(PATH + 'task.csv')
tasks.head()

,taskId,createdDate,updatedDate,name,descriptionUrl,description,verification,githubPrRequired,useJury,allowStudentArtefacts,...,alias,completed,descriptionCourse,descriptionUrlCourse,planned,courseStartDate,courseEndDate,fullName,registrationEndDate,inviteOnly
0,413,2019-08-29 10:57:34.732592,2019-11-11 18:19:01.013044,ST JS Test,http://learn.javascript.ru/,NaN,manual,f,f,f,...,rs-st-2019-q1,t,RS Short Track 2019 Q1,NaN,f,2019-04-08 07:23:45.643+00,2019-07-10 07:23:45.643+00,Rolling Scopes School 2019 Q1: Short Track,NaN,t
1,253,2019-05-17 13:02:30.233767,2019-05-17 13:02:30.233767,tdd,NaN,NaN,manual,NaN,f,f,...,rs-st-2019-q1,t,RS Short Track 2019 Q1,NaN,f,2019-04-08 07:23:45.643+00,2019-07-10 07:23:45.643+00,Rolling Scopes School 2019 Q1: Short Track,NaN,t
2,410,2019-08-29 09:41:00.400898,2019-08-29 10:08:08.993969,ST Chat,https://github.com/rolling-scopes-school/RS-Sh...,NaN,manual,t,f,f,...,rs-st-2019-q1,t,RS Short Track 2019 Q1,NaN,f,2019-04-08 07:23:45.643+00,2019-07-10 07:23:45.643+00,Rolling Scopes School 2019 Q1: Short Track,NaN,t
3,407,2019-08-29 09:32:17.606001,2019-08-29 10:08:44.864627,ST Custom Lodash,https://github.com/rolling-scopes-school/RS-Sh...,NaN,manual,t,f,f,...,rs-st-2019-q1,t,RS Short Track 2019 Q1,NaN,f,2019-04-08 07:23:45.643+00,2019-07-10 07:23:45.643+00,Rolling Scopes School 2019 Q1: Short Track,NaN,t
4,230,2019-05-17 13:01:38.681206,2019-08-29 10:10:10.985834,ST JS Assignments,https://github.com/rolling-scopes-school/RS-Sh...,NaN,manual,t,f,f,...,rs-st-2019-q1,t,RS Short Track 2019 Q1,NaN,f,2019-04-08 07:23:45.643+00,2019-07-10 07:23:45.643+00,Rolling Scopes School 2019 Q1: Short Track,NaN,t


In [11]:
tasks[['taskId', 'courseId']].isna().mean().round(4) * 100

taskId      0.0
courseId    0.0
dtype: float64

In [12]:
task_results = pd.read_csv(PATH + 'task_results.csv')
task_results['taskId'] = task_results['courseTaskId']
print(len(task_results))
task_results.head()

94262


,createdDate,updatedDate,score,comment,studentId,courseTaskId,historicalScores,juryScores,checkerId,anonymous,taskId
0,2019-09-19 18:36:46.232791,2019-09-19 18:36:46.232791,100,NaN,7266,300,"[{""authorId"":0,""score"":100,""dateTime"":15689182...",[],NaN,NaN,300
1,2019-10-02 12:48:27.977059,2019-10-21 09:58:59.674519,41,NaN,7929,313,"[{""authorId"":0,""score"":38,""dateTime"":157002050...",[],NaN,NaN,313
2,2019-10-08 15:05:43.777805,2019-10-08 15:05:43.777805,100,NaN,6134,308,"[{""authorId"":0,""score"":100,""dateTime"":15705471...",[],NaN,NaN,308
3,2019-10-11 15:03:28.245927,2019-10-11 15:03:28.245927,100,NaN,5882,305,"[{""authorId"":0,""score"":100,""dateTime"":15708062...",[],NaN,NaN,305
4,2019-10-12 16:31:57.699719,2019-10-12 16:31:57.699719,100,NaN,8247,305,"[{""authorId"":0,""score"":100,""dateTime"":15708979...",[],NaN,NaN,305


In [13]:
task_results['taskId'].isna().mean().round(4) * 100

0.0

### Combine task results with tasks data

In [14]:
task_results_enriched_with_task = pd.merge(task_results, tasks, how = 'left', on=["taskId"],
                          suffixes=("_tasks", "_results"))

In [15]:
task_results_enriched_with_task[['taskId', 'courseId']].isna().mean().round(4) * 100

taskId       0.00
courseId    53.04
dtype: float64

In [16]:
print(len(task_results_enriched_with_task))
task_results_enriched_with_task = task_results_enriched_with_task[task_results_enriched_with_task['courseId'].notnull()]
print(len(task_results_enriched_with_task))
task_results_enriched_with_task['courseId'].isna().mean().round(4) * 100

108005
50714


0.0

### Merge tasks and courses data

In [23]:
task_results_enriched_with_course = pd.merge(task_results_enriched_with_task, courses, how = 'left', on=['courseId'],
                          suffixes=("_tasks", "_courses"))
task_results_enriched_with_course.head()

,createdDate_tasks,updatedDate_tasks,score,comment,studentId,courseTaskId_tasks,historicalScores,juryScores,checkerId,anonymous,...,alias_courses,completed_courses,description_courses,descriptionUrl_courses,planned_courses,courseStartDate_courses,courseEndDate_courses,fullName_courses,registrationEndDate_courses,inviteOnly_courses
0,2020-04-30 12:29:05.513186,2020-04-30 12:29:05.513186,0,Not done. Total is 0. Impossible to leave deta...,9305,433,"[{""authorId"":2251,""score"":0,""dateTime"":1588249...",[],NaN,NaN,...,rs-2019-q3,t,RS 2019 Q3,NaN,f,2019-09-09 07:35:20.981+00,2020-01-31 07:35:20.981+00,Rolling Scopes School 2019 Q3,NaN,f
1,2020-04-30 12:29:05.513186,2020-04-30 12:29:05.513186,0,Not done. Total is 0. Impossible to leave deta...,9305,433,"[{""authorId"":2251,""score"":0,""dateTime"":1588249...",[],NaN,NaN,...,rs-2020-q1,t,Javascript / Frontend Курс.\nВводное занятие -...,NaN,f,2020-02-02 09:01:56.398+00,2020-07-31 08:01:56.398+00,Rolling Scopes School 2020 Q1: JavaScript/Fron...,2020-03-31 08:40:46.24+00,f
2,2020-05-03 19:38:39.705228,2020-05-03 19:38:39.705228,25,NaN,10651,449,"[{""authorId"":2084,""score"":25,""dateTime"":158853...",[],NaN,NaN,...,st-2019-q3,t,Short Track 2019Q3,NaN,f,2019-10-28 09:59:47.095+00,2020-01-31 09:59:47.095+00,Rolling Scopes School: Short Track 2019 Q3,2019-10-23 10:00:06.576+00,t
3,2020-05-03 19:38:39.74953,2020-05-03 19:38:39.74953,25,NaN,9137,449,"[{""authorId"":2084,""score"":25,""dateTime"":158853...",[],NaN,NaN,...,st-2019-q3,t,Short Track 2019Q3,NaN,f,2019-10-28 09:59:47.095+00,2020-01-31 09:59:47.095+00,Rolling Scopes School: Short Track 2019 Q3,2019-10-23 10:00:06.576+00,t
4,2020-05-03 19:38:39.797735,2020-05-03 19:38:39.797735,21,NaN,9053,449,"[{""authorId"":2084,""score"":21,""dateTime"":158853...",[],NaN,NaN,...,st-2019-q3,t,Short Track 2019Q3,NaN,f,2019-10-28 09:59:47.095+00,2020-01-31 09:59:47.095+00,Rolling Scopes School: Short Track 2019 Q3,2019-10-23 10:00:06.576+00,t


In [24]:
task_results_enriched_with_course.columns

Index(['createdDate_tasks', 'updatedDate_tasks', 'score', 'comment',
       'studentId', 'courseTaskId_tasks', 'historicalScores', 'juryScores',
       'checkerId', 'anonymous', 'taskId', 'createdDate_results',
       'updatedDate_results', 'name_tasks', 'descriptionUrl_tasks',
       'description_tasks', 'verification', 'githubPrRequired', 'useJury',
       'allowStudentArtefacts', 'githubRepoName', 'sourceGithubRepoUrl',
       'type', 'tags', 'discipline', 'attributes', 'courseTaskId_results',
       'createdDateCourseTask', 'updatedDateCourseTask', 'mentorStartDate',
       'mentorEndDate', 'maxScore', 'stageId', 'scoreWeight', 'checker',
       'taskOwnerId', 'studentStartDate', 'studentEndDate', 'courseId',
       'pairsCount', 'typeCourseTask', 'disabled', 'createdDateCourse',
       'updatedDateCourse', 'nameCourse', 'year_tasks', 'primarySkillId_tasks',
       'primarySkillName_tasks', 'locationName_tasks', 'alias_tasks',
       'completed_tasks', 'descriptionCourse', 'descrip

In [25]:
task_results_enriched_with_course['weighted_score'] = task_results_enriched_with_course['scoreWeight'] * task_results_enriched_with_course['score']
task_results_enriched_with_course['weighted_maxScore'] = task_results_enriched_with_course['score'] / task_results_enriched_with_course['maxScore']


In [26]:
task_results_enriched_with_course.isna().mean().round(4) * 100

createdDate_tasks               0.00
updatedDate_tasks               0.00
score                           0.00
comment                        27.14
studentId                       0.00
                               ...  
fullName_courses                0.00
registrationEndDate_courses    65.74
inviteOnly_courses              0.00
weighted_score                  0.00
weighted_maxScore              17.47
Length: 78, dtype: float64

In [27]:
subset_2 = task_results_enriched_with_course[[
    'courseId', 'studentId',
    'weighted_score', 'scoreWeight', 'score', 'maxScore', 'weighted_maxScore']]
subset_2.isna().mean().round(4) * 100

courseId              0.00
studentId             0.00
weighted_score        0.00
scoreWeight           0.00
score                 0.00
maxScore             17.47
weighted_maxScore    17.47
dtype: float64

In [28]:
task_results_enriched_subset = task_results_enriched_with_course[[
    'courseId', 'studentId',
    'weighted_score',
    'checkerId', 'checker', 'type', 'typeCourseTask', 'maxScore', 
    'primarySkillId_tasks', 'locationName_courses', 'inviteOnly_tasks',
    'pairsCount', 'taskOwnerId', 'year_courses', 
    'verification', 'githubPrRequired', 'useJury', 'allowStudentArtefacts']]

### Combine task grouped by corse

In [29]:
from scipy import stats

task_results_combined_by_course_0 = task_results_enriched_subset.groupby(['courseId', 'studentId', ]) \
       .agg({'weighted_score': 'sum',
            'pairsCount': 'median',
            'checkerId': lambda x: stats.mode(x)[0],
            'checker': lambda x: stats.mode(x)[0],
            'verification': lambda x: stats.mode(x)[0],
            'githubPrRequired': lambda x: stats.mode(x)[0],
            'useJury': lambda x: stats.mode(x)[0],
            'allowStudentArtefacts': lambda x: stats.mode(x)[0],
            'type': lambda x: stats.mode(x)[0],
            'typeCourseTask': lambda x: stats.mode(x)[0],
            'maxScore': 'median',
            'taskOwnerId': lambda x: stats.mode(x)[0],            
            'year_courses': lambda x: stats.mode(x)[0],
            'primarySkillId_tasks': lambda x: stats.mode(x)[0],
            'locationName_courses': lambda x: stats.mode(x)[0],
            'inviteOnly_tasks': lambda x: stats.mode(x)[0]})
task_results_combined_by_course_0 = task_results_combined_by_course_0.reset_index()
task_results_combined_by_course_0.head()

,courseId,studentId,weighted_score,pairsCount,checkerId,checker,verification,githubPrRequired,useJury,allowStudentArtefacts,type,typeCourseTask,maxScore,taskOwnerId,year_courses,primarySkillId_tasks,locationName_courses,inviteOnly_tasks
0,1.0,5706,815.0,NaN,NaN,mentor,manual,f,f,f,codejam,0,55.0,NaN,2019.0,javascript,Minsk,f
1,1.0,5707,2470.0,NaN,NaN,mentor,manual,f,f,f,codejam,0,55.0,NaN,2019.0,javascript,Minsk,f
2,1.0,5709,2705.0,NaN,NaN,mentor,manual,f,f,f,codejam,0,55.0,NaN,2019.0,javascript,Minsk,f
3,1.0,5712,280.0,NaN,NaN,mentor,manual,f,f,f,codejam,0,100.0,NaN,2019.0,javascript,Minsk,f
4,1.0,5713,1340.0,NaN,NaN,mentor,manual,f,f,f,codejam,0,55.0,NaN,2019.0,javascript,Minsk,f


In [30]:
task_results_combined_by_course_0.isna().mean().round(4) * 100

courseId                  0.00
studentId                 0.00
weighted_score            0.00
pairsCount               90.30
checkerId                75.95
checker                   0.00
verification              0.00
githubPrRequired          0.00
useJury                   0.00
allowStudentArtefacts     0.00
type                      0.00
typeCourseTask            0.00
maxScore                 39.74
taskOwnerId              61.59
year_courses             56.25
primarySkillId_tasks      0.00
locationName_courses      0.00
inviteOnly_tasks          0.00
dtype: float64

In [31]:
task_results_combined_by_course = task_results_combined_by_course_0.drop(
    columns=['pairsCount', 'checkerId', 'taskOwnerId', 'year_courses', 'maxScore',
            'checker', 'verification', 'typeCourseTask'])

task_results_combined_by_course.isna().mean().round(4) * 100

courseId                 0.0
studentId                0.0
weighted_score           0.0
githubPrRequired         0.0
useJury                  0.0
allowStudentArtefacts    0.0
type                     0.0
primarySkillId_tasks     0.0
locationName_courses     0.0
inviteOnly_tasks         0.0
dtype: float64

### Apply StandardScaler to 'weighted_score'

In [26]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['weighted_score'])])

#transformed_column = preprocessor.fit_transform(task_results_combined_by_course)
#task_results_combined_by_course['weighted_score'] = transformed_column
#task_results_combined_by_course.head()

### Apply OneHotEncoder to categorical columns

In [27]:
#task_results_combined_by_course = pd.get_dummies(task_results_combined_by_course, columns=['checker', 'verification', 'typeCourseTask'])
#task_results_combined_by_course.head(0)

# Preprocess data

In [42]:
def convert_boolean(x):
    if x == True:
        return 1
    elif x == False:
        return 0
    elif x == 't':
        return 1
    elif x == 'f':
        return 0
    elif x == 'True':
        return 1
    elif x == 'False':
        return 0
    else:
        return x

convert_boolean_vector = np.vectorize(convert_boolean)
    


def preprocess_data(row_data):
    
    students_data = row_data.copy()
    
    # Merge task results data with students data
    students_courses_enriched = pd.merge(students_data, task_results_combined_by_course, how = 'left', 
                                          on=['courseId', 'studentId'],
                                          suffixes=("_students", "_courses"))
    
    # remove columns
    students_courses_enriched = students_courses_enriched.drop(
    columns=['expellingReason', 'preferedMentorGithubId', 'readyFullTime', 'cvUrl', 'hiredById', 'hiredByName',
             'endDate', 'repository', 'repositoryLastActivityDate', 'englishLevel','mentorId',            
            'createdDate', 'updatedDate', 'userId', 'startDate', 'idUser',
            'educationHistory', 'locationName_courses',
            'githubPrRequired', 'useJury', 'allowStudentArtefacts','inviteOnly_tasks',
            'courseId'])
    
    # Drop type, primarySkillId_tasks (it is correlated with typeCourseTask)
    students_courses_enriched = students_courses_enriched.drop(
        columns=['type', 'primarySkillId_tasks'])
    
    # Normalize data
    students_courses_enriched = students_courses_enriched.apply(convert_boolean_vector)
    
    # fill empty score
    values = {'weighted_score': 0}
    students_courses_enriched = students_courses_enriched.fillna(value=values)
    
    # Apply StandardScaler to 'weighted_score'
    preprocessor = ColumnTransformer(transformers=[('num', StandardScaler(), ['weighted_score'])])
    transformed_column = preprocessor.fit_transform(students_courses_enriched)
    students_courses_enriched['weighted_score'] = transformed_column
    
    # Categorize courseId
    # students_courses_enriched = pd.get_dummies(students_courses_enriched, columns=['courseId'])
    
    return students_courses_enriched
    

In [43]:
train_data = preprocess_data(students_courses)

train_data.isna().mean().round(4) * 100

studentId          0.0
isExpelled         0.0
courseCompleted    0.0
isTopPerformer     0.0
isFailed           0.0
weighted_score     0.0
dtype: float64

In [44]:
train_data.head(15)

,studentId,isExpelled,courseCompleted,isTopPerformer,isFailed,weighted_score
0,5740,1,0,1,0,-0.207666
1,6956,1,0,1,0,-0.207666
2,11616,0,0,0,0,-0.207666
3,11773,0,0,0,0,-0.207666
4,6724,1,0,1,0,-0.207666
5,7409,0,0,1,0,-0.207666
6,6820,0,0,1,0,-0.207666
7,12326,0,0,1,0,4.090759
8,13609,1,0,1,0,-0.207666
9,9544,0,0,1,0,6.423365


In [45]:
# Seperate classes
target_0 = train_data[train_data['isTopPerformer'] == 0]
target_1 = train_data[train_data['isTopPerformer'] == 1]
print(len(target_0), len(target_1))

7892 2579


In [46]:
target_0.isna().mean().round(4) * 100

studentId          0.0
isExpelled         0.0
courseCompleted    0.0
isTopPerformer     0.0
isFailed           0.0
weighted_score     0.0
dtype: float64

In [47]:
target_1.isna().mean().round(4) * 100

studentId          0.0
isExpelled         0.0
courseCompleted    0.0
isTopPerformer     0.0
isFailed           0.0
weighted_score     0.0
dtype: float64

# First experiment - balancing classifier model

In [156]:
# Code for calculating Normalized gini coefficient
# https://www.kaggle.com/c/ClaimPredictionChallenge/discussion/703
def gini(actual, pred, cmpcol = 0, sortcol = 1):  
    assert(len(actual) == len(pred))  
    epsilon = 1e-7
    values = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)  
    values = values[np.lexsort((values[:, 2], -1 * values[:, 1]))]  
    total = values[:, 0].sum() 
    gini_sum = (values[:, 0].cumsum().sum() + epsilon) / (total + epsilon)  
  
    gini_sum -= (len(actual) + 1) / 2  
    return gini_sum / len(actual)  
  
def gini_normalized(a, p):  
    '''Function to calculate the normalized gini coefficient'''
    return gini(a, p) / gini(a, a)

def evaluate_performance(predictions, test_labels):
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [148]:
X = train_data.drop(columns=['isTopPerformer', 'studentId'])
y = train_data['isTopPerformer']

In [149]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020)

In [150]:
pipeline = Pipeline(steps=[
                           ('classifier', SGDClassifier(loss='log', alpha=0.001, n_jobs=-1, random_state=14,
                                                       class_weight='balanced'))])

clf_initial_1 = pipeline.fit(X_train, y_train)

In [157]:
y_pred = clf_initial_1.predict_proba(X_test)[:, 1]

print('Accuracy: ' + str(accuracy_score(y_true=y_test, y_pred=clf_initial_1.predict(X_test))))
print('Gini: ' + str(gini_normalized(y_test, y_pred)))
evaluate_performance(y_pred, y_test)

Accuracy: 0.7651551312649165
Gini: 0.6096277287230971
Model Performance
Average Error: 0.3734 degrees.
Accuracy = -inf%.


-inf

### Try to remove some features

In [162]:
X = train_data.drop(columns=['isTopPerformer', 'studentId',
'isFailed'])
y = train_data['isTopPerformer']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020)

pipeline = Pipeline(steps=[
                           ('classifier', SGDClassifier(loss='log', alpha=0.001, n_jobs=-1, random_state=14,
                                                       class_weight='balanced'))])

clf_initial_2 = pipeline.fit(X_train, y_train)

y_pred = clf_initial_2.predict_proba(X_test)[:, 1]

print('Accuracy: ' + str(accuracy_score(y_true=y_test, y_pred=clf_initial_2.predict(X_test))))
print('Gini: ' + str(gini_normalized(y_test, y_pred)))
evaluate_performance(y_pred, y_test)

Accuracy: 0.7651551312649165
Gini: 0.6020805684125725
Model Performance
Average Error: 0.3741 degrees.
Accuracy = -inf%.


-inf

In [158]:
train_data.columns

Index(['studentId', 'isExpelled', 'courseCompleted', 'isTopPerformer',
       'isFailed', 'weighted_score'],
      dtype='object')

# Analyze for oversampling and undersampling

In [54]:
# Seperate classes
target_0 = train_data[train_data['isTopPerformer'] == 0]
target_1 = train_data[train_data['isTopPerformer'] == 1]
print(len(target_0), len(target_1))

7892 2579


# Second experiment - random undersampling

In [55]:
min_count = min(len(target_0), len(target_1))
print(len(target_0), len(target_1), min_count)
# Resample
target_0_undersample = target_0.sample(min_count)
target_1_undersample = target_1.sample(min_count)
# Merge back to single df
undersample_dataset = pd.concat([target_0_undersample, target_1_undersample], 
                             axis=0)

7892 2579 2579


In [56]:
# prepare data
y_undersample = undersample_dataset['isTopPerformer']
X_undersample = undersample_dataset.drop(columns=['isTopPerformer', 'studentId'])
X_train, X_test, y_train, y_test = train_test_split(X_undersample, y_undersample, test_size=0.3, random_state=42)

In [57]:
pipeline = Pipeline(steps=[
                           ('classifier', SGDClassifier(loss='log', alpha=0.001, n_jobs=-1, random_state=14))])

clf_undersampling_1 = pipeline.fit(X_train, y_train)

y_pred = clf_undersampling_1.predict_proba(X_test)[:, 1]

print('Accuracy: ' + str(accuracy_score(y_true=y_test, y_pred=clf_undersampling_1.predict(X_test))))
print('Gini: ' + str(gini_normalized(y_test, y_pred)))

Accuracy: 0.7422480620155039
Gini: 0.5635976226682241


# Third Experiment - random oversampling

In [58]:
max_count = max(len(target_0), len(target_1))
print(len(target_0), len(target_1), max_count)

# Resample
target_0_oversample = target_0.sample(max_count, replace=True)
target_1_oversample = target_1.sample(max_count, replace=True)
# Merge back to single df
oversample_dataset = pd.concat([target_0_oversample, target_1_oversample], axis=0)

7892 2579 7892


In [59]:
# prepare data
y_oversample = oversample_dataset['isTopPerformer']
X_oversample = oversample_dataset.drop(columns=['isTopPerformer', 'studentId'])
X_train, X_test, y_train, y_test = train_test_split(X_oversample, y_oversample, test_size=0.3, random_state=42)

In [60]:
pipeline = Pipeline(steps=[('classifier', SGDClassifier(loss='log', alpha=0.001, n_jobs=-1, random_state=14))])

clf_oversampling_1 = pipeline.fit(X_train, y_train)

y_pred = clf_oversampling_1.predict_proba(X_test)[:, 1]

print('Accuracy: ' + str(accuracy_score(y_true=y_test, y_pred=clf_oversampling_1.predict(X_test))))
print('Gini: ' + str(gini_normalized(y_test, y_pred)))

Accuracy: 0.7472550675675675
Gini: 0.5791015459194674


### Load Test Data

In [61]:
students_courses_test = pd.read_csv(PATH + 'test.csv').set_index('studentId')
print(students_courses_test.columns)
students_courses_test.head()

Index(['createdDate', 'updatedDate', 'isExpelled', 'expellingReason',
       'courseCompleted', 'preferedMentorGithubId', 'readyFullTime',
       'courseId', 'userId', 'mentorId', 'cvUrl', 'hiredById', 'hiredByName',
       'isFailed', 'startDate', 'endDate', 'repository',
       'repositoryLastActivityDate', 'idUser', 'educationHistory',
       'englishLevel'],
      dtype='object')


,createdDate,updatedDate,isExpelled,expellingReason,courseCompleted,preferedMentorGithubId,readyFullTime,courseId,userId,mentorId,...,hiredById,hiredByName,isFailed,startDate,endDate,repository,repositoryLastActivityDate,idUser,educationHistory,englishLevel
studentId,,,,,,,,,,,,,,,,,,,,,
11450,2020-02-05 09:18:15.61238,2020-04-01 13:30:00.599077,f,NaN,f,NaN,NaN,16,4546,NaN,...,NaN,NaN,f,2020-02-05 09:18:15.609+00,NaN,NaN,NaN,4546,[],NaN
7855,2019-09-11 18:05:14.135267,2019-11-20 22:00:13.036321,f,NaN,f,NaN,NaN,11,6352,NaN,...,NaN,NaN,f,2019-09-09 07:35:20.981+00,NaN,NaN,NaN,6352,[],NaN
12753,2020-02-17 16:31:47.104981,2020-10-13 21:00:01.563666,f,NaN,f,NaN,NaN,19,10322,NaN,...,NaN,NaN,f,2020-02-17 16:31:47.103+00,NaN,NaN,NaN,10322,[],NaN
14717,2020-07-18 11:45:19.46207,2020-09-29 21:00:15.918095,f,NaN,f,NaN,NaN,28,12002,NaN,...,NaN,NaN,f,2020-07-18 11:45:19.459+00,NaN,NaN,NaN,12002,[],a2
7455,2019-09-10 14:01:26.166087,2020-01-20 21:20:12.461623,f,NaN,f,NaN,NaN,11,6008,951.0,...,NaN,NaN,f,2019-09-09 07:35:20.981+00,NaN,https://github.com/rolling-scopes-school/094ab...,NaN,6008,[],NaN


In [62]:
students_courses_test_preprocessed = preprocess_data(students_courses_test)

students_courses_test_preprocessed.isna().mean().round(4) * 100

studentId          0.0
isExpelled         0.0
courseCompleted    0.0
isFailed           0.0
weighted_score     0.0
dtype: float64

In [61]:
students_courses_test_preprocessed.head()

,studentId,isExpelled,courseCompleted,isFailed,weighted_score
0,11450,0,0,0,-0.172142
1,7855,0,0,0,-0.172142
2,12753,0,0,0,-0.172142
3,14717,0,0,0,-0.172142
4,7455,0,0,0,-0.172142


In [63]:
sample_submission = pd.read_csv(PATH + 'sampleSubmission.csv')
print(sample_submission.columns)
sample_submission.head()

Index(['studentId', 'isTopPerformer'], dtype='object')


,studentId,isTopPerformer
0,11450,False
1,7855,False
2,12753,False
3,14717,False
4,7455,True


# Another features

In [198]:
task_results_enriched_with_course.columns

Index(['createdDate_tasks', 'updatedDate_tasks', 'score', 'comment',
       'studentId', 'courseTaskId_tasks', 'historicalScores', 'juryScores',
       'checkerId', 'anonymous', 'taskId', 'createdDate_results',
       'updatedDate_results', 'name_tasks', 'descriptionUrl_tasks',
       'description_tasks', 'verification', 'githubPrRequired', 'useJury',
       'allowStudentArtefacts', 'githubRepoName', 'sourceGithubRepoUrl',
       'type', 'tags', 'discipline', 'attributes', 'courseTaskId_results',
       'createdDateCourseTask', 'updatedDateCourseTask', 'mentorStartDate',
       'mentorEndDate', 'maxScore', 'stageId', 'scoreWeight', 'checker',
       'taskOwnerId', 'studentStartDate', 'studentEndDate', 'courseId',
       'pairsCount', 'typeCourseTask', 'disabled', 'createdDateCourse',
       'updatedDateCourse', 'nameCourse', 'year_tasks', 'primarySkillId_tasks',
       'primarySkillName_tasks', 'locationName_tasks', 'alias_tasks',
       'completed_tasks', 'descriptionCourse', 'descrip

In [199]:
task_results_enriched_with_course.isna().mean().round(4) * 100

createdDate_tasks               0.00
updatedDate_tasks               0.00
score                           0.00
comment                        27.14
studentId                       0.00
                               ...  
fullName_courses                0.00
registrationEndDate_courses    65.74
inviteOnly_courses              0.00
weighted_score                  0.00
weighted_maxScore              17.47
Length: 78, dtype: float64

In [ ]:
task_results_enriched_subset_2 = task_results_enriched_with_course[[
    'courseId', 'studentId',
    'weighted_score',
    'checkerId', 'checker', 'type', 'typeCourseTask', 'maxScore', 
    'primarySkillId_tasks', 'locationName_courses', 'inviteOnly_tasks',
    'pairsCount', 'taskOwnerId', 'year_courses', 
    'verification', 'githubPrRequired', 'useJury', 'allowStudentArtefacts']]

In [ ]:
task_results_combined_by_course_2_0 = task_results_enriched_subset_2.groupby(['courseId', 'studentId', ]) \
       .agg({'weighted_score': 'sum',
            'pairsCount': 'median',
            'checkerId': lambda x: stats.mode(x)[0],
            'checker': lambda x: stats.mode(x)[0],
            'verification': lambda x: stats.mode(x)[0],
            'githubPrRequired': lambda x: stats.mode(x)[0],
            'useJury': lambda x: stats.mode(x)[0],
            'allowStudentArtefacts': lambda x: stats.mode(x)[0],
            'type': lambda x: stats.mode(x)[0],
            'typeCourseTask': lambda x: stats.mode(x)[0],
            'maxScore': 'median',
            'taskOwnerId': lambda x: stats.mode(x)[0],            
            'year_courses': lambda x: stats.mode(x)[0],
            'primarySkillId_tasks': lambda x: stats.mode(x)[0],
            'locationName_courses': lambda x: stats.mode(x)[0],
            'inviteOnly_tasks': lambda x: stats.mode(x)[0]})
task_results_combined_by_course_2_0 = task_results_combined_by_course_2_0.reset_index()

task_results_combined_by_course_2_0.isna().mean().round(4) * 100
task_results_combined_by_course_2_0.head()

In [ ]:
task_results_combined_by_course_2 = task_results_combined_by_course_2_0.drop(
    columns=['pairsCount', 'checkerId', 'taskOwnerId', 'year_courses', 'maxScore',
            'checker', 'verification', 'typeCourseTask'])

In [224]:

def preprocess_data_2(row_data):
    
    students_data = row_data.copy()
    
    # Merge task results data with students data
    students_courses_enriched = pd.merge(students_data, task_results_combined_by_course, how = 'left', 
                                          on=['courseId', 'studentId'],
                                          suffixes=("_students", "_courses"))
    
    # remove columns
    students_courses_enriched = students_courses_enriched.drop(
    columns=['courseId', 'endDate', 'createdDate', 'updatedDate', 'startDate', 'userId', 'idUser',
            'expellingReason', 'preferedMentorGithubId',
            'mentorId','cvUrl', 'hiredById', 'hiredByName',
            'repository', 'repositoryLastActivityDate',
            'englishLevel',  'githubPrRequired', 'useJury',
            'locationName_courses',
            'allowStudentArtefacts','inviteOnly_tasks',
            'educationHistory',
            'type', 'primarySkillId_tasks',
            'readyFullTime',
            'isFailed'])

    
    # Normalize data
    students_courses_enriched = students_courses_enriched.apply(convert_boolean_vector)
    
    # fill empty score
    values = {'weighted_score': 0}
    students_courses_enriched = students_courses_enriched.fillna(value=values)
    
    # Apply StandardScaler to 'weighted_score'
    preprocessor = ColumnTransformer(transformers=[('num', StandardScaler(), ['weighted_score'])])
    transformed_column = preprocessor.fit_transform(students_courses_enriched)
    students_courses_enriched['weighted_score'] = transformed_column
    
    # Categorize courseId
    # students_courses_enriched = pd.get_dummies(students_courses_enriched, columns=['courseId'])
    
    return students_courses_enriched
    

In [225]:
train_data_2 = preprocess_data_2(students_courses)

train_data_2.isna().mean().round(4) * 100

studentId          0.0
isExpelled         0.0
courseCompleted    0.0
isTopPerformer     0.0
weighted_score     0.0
dtype: float64

In [226]:
train_data_2.head()

,studentId,isExpelled,courseCompleted,isTopPerformer,weighted_score
0,5740,1,0,1,-0.207666
1,6956,1,0,1,-0.207666
2,11616,0,0,0,-0.207666
3,11773,0,0,0,-0.207666
4,6724,1,0,1,-0.207666


In [227]:
X = train_data_2.drop(columns=['isTopPerformer', 'studentId'])
y = train_data_2['isTopPerformer']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020)

pipeline = Pipeline(steps=[
                           ('classifier', SGDClassifier(loss='log', alpha=0.001, n_jobs=-1, random_state=14,
                                                       class_weight='balanced'))])

clf_initial_2 = pipeline.fit(X_train, y_train)

y_pred = clf_initial_2.predict_proba(X_test)[:, 1]

print('Accuracy: ' + str(accuracy_score(y_true=y_test, y_pred=clf_initial_2.predict(X_test))))
print('Gini: ' + str(gini_normalized(y_test, y_pred)))
evaluate_performance(y_pred, y_test)

Accuracy: 0.7651551312649165
Gini: 0.6020805684125725
Model Performance
Average Error: 0.3741 degrees.
Accuracy = -inf%.


-inf

# Kaggle

In [237]:
def generate_results_for_kaggle(raw_data, test_data, clf_model, file_name): 
    
    data = test_data.copy()
    pred_on_test_data = clf_model.predict_proba(data)[:, 1]
    
    data['isTopPerformer_value'] = pred_on_test_data
    data['isTopPerformer'] = data['isTopPerformer_value'] > 0.95
    data['studentId'] = raw_data['studentId']
    

    """
    The file should have exactly 2 columns: studentId, isTopPerformer
    Submission must have 2991
    """
    sub_set = data[['studentId', 'isTopPerformer']]
    sub_set.to_csv(file_name, index=False)
    
    return data

In [238]:
# remove some features
students_courses_test_preprocessed_2 = preprocess_data_2(students_courses_test)
test_data_2 = students_courses_test_preprocessed_2.drop(columns=['studentId'])


result1=generate_results_for_kaggle(students_courses_test_preprocessed, test_data_2, clf_initial_2, "clf_initial_2_sum_without_isFailed.csv")


In [239]:
# Seperate classes
target_0 = result1[result1['isTopPerformer'] == False]
target_1 = result1[result1['isTopPerformer'] == True]
print(len(target_0), len(target_1))

2902 89


In [241]:
result1.head(-40)

,isExpelled,courseCompleted,weighted_score,isTopPerformer_value,isTopPerformer,studentId
0,0,0,-0.166191,0.706793,False,11450
1,0,0,-0.166191,0.706793,False,7855
2,0,0,-0.166191,0.706793,False,12753
3,0,0,-0.166191,0.706793,False,14717
4,0,0,-0.166191,0.706793,False,7455
...,...,...,...,...,...,...
2946,0,0,-0.166191,0.706793,False,15423
2947,0,0,-0.166191,0.706793,False,15086
2948,0,0,-0.166191,0.706793,False,14532
2949,0,0,-0.166191,0.706793,False,14771


In [76]:
# remove some features
test_data = students_courses_test_preprocessed.drop(columns=['studentId'])

result1=generate_results_for_kaggle(students_courses_test_preprocessed, test_data, clf_initial_1, "clf_initial_1_sum.csv")
result2=generate_results_for_kaggle(students_courses_test_preprocessed, test_data, clf_undersampling_1, "clf_undersampling_1_sum.csv")
result3=generate_results_for_kaggle(students_courses_test_preprocessed, test_data, clf_oversampling_1, "clf_oversampling_1_sum.csv")


In [77]:
# Seperate classes
target_0 = result1[result1['isTopPerformer'] == False]
target_1 = result1[result1['isTopPerformer'] == True]
print(len(target_0), len(target_1))

target_0 = result2[result2['isTopPerformer'] == False]
target_1 = result2[result2['isTopPerformer'] == True]
print(len(target_0), len(target_1))

target_0 = result3[result3['isTopPerformer'] == False]
target_1 = result3[result3['isTopPerformer'] == True]
print(len(target_0), len(target_1))

2865 126
2869 122
2869 122


In [78]:
sample_submission = pd.read_csv(PATH + 'sampleSubmission.csv')
print(sample_submission.columns)
sample_submission.head()

Index(['studentId', 'isTopPerformer'], dtype='object')


,studentId,isTopPerformer
0,11450,False
1,7855,False
2,12753,False
3,14717,False
4,7455,True


In [80]:
my_submission = pd.read_csv('clf_initial_1_sum.csv')
print(my_submission.columns)
print(len(my_submission))
my_submission.head()

Index(['studentId', 'isTopPerformer'], dtype='object')
2991


,studentId,isTopPerformer
0,11450,False
1,7855,False
2,12753,False
3,14717,False
4,7455,False


# AgglomerativeClustering с использованием TSNE трансформации

In [82]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score

X = test_data


tsne_transformer = TSNE(n_components = 2)
transformed_X = tsne_transformer.fit_transform(X)

clustering = AgglomerativeClustering(n_clusters=2)
labels = clustering.fit_predict(X)

silhouette_scores_original = silhouette_score(X, labels)   
#v_measure_score_original = v_measure_score(y, labels)

labels_transformed = clustering.fit_predict(transformed_X)
silhouette_scores_transformed = silhouette_score(transformed_X, labels_transformed)   
#v_measure_score_transformed = v_measure_score(y, labels_transformed)

print(f'Metrics on original data: silhouette {silhouette_scores_original}')
print(f'Metrics on transformed data: silhouette {silhouette_scores_transformed}')

Metrics on original data: silhouette 0.9821863358863359
Metrics on transformed data: silhouette 0.9091189503669739


In [89]:
def generate_results_for_kaggle_2(raw_data, test_data, predicted_labels, file_name): 
    
    data = test_data.copy()
    
    data['isTopPerformer_value'] = predicted_labels
    data['isTopPerformer'] = data['isTopPerformer_value'] == 0
    data['studentId'] = raw_data['studentId']
    

    """
    The file should have exactly 2 columns: studentId, isTopPerformer
    Submission must have 2991
    """
    sub_set = data[['studentId', 'isTopPerformer']]
    sub_set.to_csv(file_name, index=False)
    
    return data

In [90]:
result=generate_results_for_kaggle_2(students_courses_test_preprocessed, test_data, labels, "AgglomerativeClustering_sum.csv")

In [91]:
result.head(30)

,isExpelled,courseCompleted,isFailed,weighted_score,isTopPerformer_value,isTopPerformer,studentId
0,0,0,0,-0.166191,1,False,11450
1,0,0,0,-0.166191,1,False,7855
2,0,0,0,-0.166191,1,False,12753
3,0,0,0,-0.166191,1,False,14717
4,0,0,0,-0.166191,1,False,7455
5,0,0,0,-0.166191,1,False,6958
6,0,0,0,-0.166191,1,False,8004
7,0,0,0,-0.166191,1,False,8575
8,0,0,0,-0.166191,1,False,15668
9,0,0,0,2.060376,0,True,9155


In [92]:
target_0 = result[result['isTopPerformer'] == False]
target_1 = result[result['isTopPerformer'] == True]
print(len(target_0), len(target_1))

2907 84


In [93]:
my_submission = pd.read_csv('AgglomerativeClustering_sum.csv')
print(my_submission.columns)
print(len(my_submission))
my_submission.head()

Index(['studentId', 'isTopPerformer'], dtype='object')
2991


,studentId,isTopPerformer
0,11450,False
1,7855,False
2,12753,False
3,14717,False
4,7455,False


# RandomForestClassifier

In [101]:
def generate_results_for_kaggle_3(raw_data, test_data, predicted_labels, file_name): 
    
    data = test_data.copy()
    
    data['isTopPerformer_value'] = predicted_labels
    data['isTopPerformer'] = data['isTopPerformer_value'] == 1
    data['studentId'] = raw_data['studentId']
    

    """
    The file should have exactly 2 columns: studentId, isTopPerformer
    Submission must have 2991
    """
    sub_set = data[['studentId', 'isTopPerformer']]
    sub_set.to_csv(file_name, index=False)
    
    return data

In [129]:
X = train_data.drop(columns=['isTopPerformer', 'studentId'])
y = train_data['isTopPerformer']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020)

In [130]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import v_measure_score

# small subset of params because of big working time
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
param_grid = { 
    'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
    'criterion': ['entropy', 'gini', 'mse'],
    'max_depth': max_depth,
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'bootstrap': [True, False]
}

model = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = model, param_distributions = param_grid, n_iter = 500, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train) 

print(rf_random.best_params_)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


C:\INST\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.81327603 0.81315664 0.81506686 0.81506686 0.81303725 0.81315664
        nan 0.81255969 0.81327603        nan 0.81303725 0.81482808
 0.81411175 0.8145893  0.81446991 0.81303725 0.81506686 0.8145893
 0.81303725 0.81387297 0.81494747        nan        nan 0.81435053
 0.81315664 0.81446991 0.81494747 0.81327603 0.81530564 0.81387297
        nan 0.81530564 0.81482808 0.81518625 0.81315664 0.81506686
        nan 0.81446991 0.81494747        nan 0.81494747 0.81446991
 0.81446991 0.81446991        nan        nan        nan 0.81518625
 0.81506686 0.81435053 0.81506686        nan 0.81446991 0.81315664
 0.81303725 0.81387297 0.81423114 0.81494747 0.81399236 0.81494747
        nan 0.81303725 0.81423114 0.81494747        nan        nan
 0.81303725 0.81363419 0.81375358 0.81411175 0.81494747        nan
 0.81303725 0.81387297        nan 0.81435053        nan      

{'n_estimators': 800, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_depth': 30, 'criterion': 'gini', 'bootstrap': True}


In [131]:
best_random_model = rf_random.best_estimator_

y_pred_train = best_random_model.predict(X_train)
print(f"Accuracy score on train set is: {accuracy_score(y_train, y_pred_train)}")
print(f"v_measure on train set is: {v_measure_score(y_train, y_pred_train)}")
evaluate_performance(y_pred_train, y_train)

y_pred = best_random_model.predict(X_test)
print(f"Accuracy score on test set is: {accuracy_score(y_test, y_pred)}")
print(f"v_measure on test set is: {v_measure_score(y_test, y_pred)}")
evaluate_performance(y_pred, y_test)

Accuracy score on train set is: 0.8157831900668577
v_measure on train set is: 0.20387769198376116
Model Performance
Average Error: 0.1842 degrees.
Accuracy = -inf%.
Accuracy score on test set is: 0.8410501193317422
v_measure on test set is: 0.26671937920449246
Model Performance
Average Error: 0.1589 degrees.
Accuracy = -inf%.


-inf

In [132]:
labels_random_forest = best_random_model.predict(test_data)

In [133]:
result_random_forest=generate_results_for_kaggle_3(students_courses_test_preprocessed, test_data, labels_random_forest, "RandomForestClassifier_sum.csv")

In [134]:
target_0 = result_random_forest[result_random_forest['isTopPerformer'] == False]
target_1 = result_random_forest[result_random_forest['isTopPerformer'] == True]
print(len(target_0), len(target_1))
result_random_forest.head(15)

0 2991


,isExpelled,courseCompleted,isFailed,weighted_score,isTopPerformer_value,isTopPerformer,studentId
0,0,0,0,-0.166191,1,True,11450
1,0,0,0,-0.166191,1,True,7855
2,0,0,0,-0.166191,1,True,12753
3,0,0,0,-0.166191,1,True,14717
4,0,0,0,-0.166191,1,True,7455
5,0,0,0,-0.166191,1,True,6958
6,0,0,0,-0.166191,1,True,8004
7,0,0,0,-0.166191,1,True,8575
8,0,0,0,-0.166191,1,True,15668
9,0,0,0,2.060376,1,True,9155


In [135]:
my_submission = pd.read_csv('RandomForestClassifier_sum.csv')
print(my_submission.columns)
print(len(my_submission))
my_submission.head()

Index(['studentId', 'isTopPerformer'], dtype='object')
2991


,studentId,isTopPerformer
0,11450,True
1,7855,True
2,12753,True
3,14717,True
4,7455,True


#### oversampling

In [136]:
# Seperate classes
target_0 = train_data[train_data['isTopPerformer'] == 0]
target_1 = train_data[train_data['isTopPerformer'] == 1]
max_count = max(len(target_0), len(target_1))
print(len(target_0), len(target_1), max_count)

# Resample
target_0_oversample = target_0.sample(max_count, replace=True)
target_1_oversample = target_1.sample(max_count, replace=True)
# Merge back to single df
oversample_dataset = pd.concat([target_0_oversample, target_1_oversample], axis=0)

# prepare data
y_oversample = oversample_dataset['isTopPerformer']
X_oversample = oversample_dataset.drop(columns=['isTopPerformer', 'studentId'])
X_train, X_test, y_train, y_test = train_test_split(X_oversample, y_oversample, test_size=0.3, random_state=42)

7892 2579 7892


In [138]:
model_oversampling = RandomForestClassifier(
    n_estimators = 800,
    min_samples_split=2,
    min_samples_leaf=4,
    max_depth=30,
    criterion='gini')
model_oversampling.fit(X_train, y_train) 

RandomForestClassifier(max_depth=30, min_samples_leaf=4, n_estimators=800)

In [139]:
y_pred_train = model_oversampling.predict(X_train)
print(f"Accuracy score on train set is: {accuracy_score(y_train, y_pred_train)}")
print(f"v_measure on train set is: {v_measure_score(y_train, y_pred_train)}")
evaluate_performance(y_pred_train, y_train)

y_pred = model_oversampling.predict(X_test)
print(f"Accuracy score on test set is: {accuracy_score(y_test, y_pred)}")
print(f"v_measure on test set is: {v_measure_score(y_test, y_pred)}")
evaluate_performance(y_pred, y_test)

Accuracy score on train set is: 0.7617668356263577
v_measure on train set is: 0.20791542412822347
Model Performance
Average Error: 0.2382 degrees.
Accuracy = -inf%.
Accuracy score on test set is: 0.7725929054054054
v_measure on test set is: 0.22658043140108172
Model Performance
Average Error: 0.2274 degrees.
Accuracy = -inf%.


-inf

In [140]:
labels_oversampling = model_oversampling.predict(test_data)
result_oversampling=generate_results_for_kaggle_3(students_courses_test_preprocessed, test_data, 
                                                  labels_oversampling, "RandomForestClassifier_sum_oversampling.csv")


target_0 = result_oversampling[result_oversampling['isTopPerformer'] == False]
target_1 = result_oversampling[result_oversampling['isTopPerformer'] == True]
print(len(target_0), len(target_1))
result_oversampling.head(15)

0 2991


,isExpelled,courseCompleted,isFailed,weighted_score,isTopPerformer_value,isTopPerformer,studentId
0,0,0,0,-0.166191,1,True,11450
1,0,0,0,-0.166191,1,True,7855
2,0,0,0,-0.166191,1,True,12753
3,0,0,0,-0.166191,1,True,14717
4,0,0,0,-0.166191,1,True,7455
5,0,0,0,-0.166191,1,True,6958
6,0,0,0,-0.166191,1,True,8004
7,0,0,0,-0.166191,1,True,8575
8,0,0,0,-0.166191,1,True,15668
9,0,0,0,2.060376,1,True,9155


In [141]:
my_submission = pd.read_csv('RandomForestClassifier_sum_oversampling.csv')
print(my_submission.columns)
print(len(my_submission))
my_submission.head()

Index(['studentId', 'isTopPerformer'], dtype='object')
2991


,studentId,isTopPerformer
0,11450,True
1,7855,True
2,12753,True
3,14717,True
4,7455,True


# undersampling

In [143]:
target_0 = train_data[train_data['isTopPerformer'] == 0]
target_1 = train_data[train_data['isTopPerformer'] == 1]
min_count = min(len(target_0), len(target_1))
print(len(target_0), len(target_1), min_count)

# Resample
target_0_undersample = target_0.sample(min_count)
target_1_undersample = target_1.sample(min_count)
# Merge back to single df
undersample_dataset = pd.concat([target_0_undersample, target_1_undersample], 
                             axis=0)

# prepare data
y_undersample = undersample_dataset['isTopPerformer']
X_undersample = undersample_dataset.drop(columns=['isTopPerformer', 'studentId'])
X_train, X_test, y_train, y_test = train_test_split(X_undersample, y_undersample, test_size=0.3, random_state=42)

7892 2579 2579


In [144]:
model_undersampling = RandomForestClassifier(
    n_estimators = 800,
    min_samples_split=2,
    min_samples_leaf=4,
    max_depth=30,
    criterion='gini')
model_undersampling.fit(X_train, y_train) 

RandomForestClassifier(max_depth=30, min_samples_leaf=4, n_estimators=800)

In [145]:
y_pred_train = model_undersampling.predict(X_train)
print(f"Accuracy score on train set is: {accuracy_score(y_train, y_pred_train)}")
print(f"v_measure on train set is: {v_measure_score(y_train, y_pred_train)}")
evaluate_performance(y_pred_train, y_train)

y_pred = model_undersampling.predict(X_test)
print(f"Accuracy score on test set is: {accuracy_score(y_test, y_pred)}")
print(f"v_measure on test set is: {v_measure_score(y_test, y_pred)}")
evaluate_performance(y_pred, y_test)

Accuracy score on train set is: 0.7684210526315789
v_measure on train set is: 0.2194280094166824
Model Performance
Average Error: 0.2316 degrees.
Accuracy = -inf%.
Accuracy score on test set is: 0.7487080103359173
v_measure on test set is: 0.18651556961947294
Model Performance
Average Error: 0.2513 degrees.
Accuracy = -inf%.


-inf

In [146]:
labels_undersampling = model_undersampling.predict(test_data)
result_undersampling=generate_results_for_kaggle_3(students_courses_test_preprocessed, test_data, 
                                                  labels_undersampling, "RandomForestClassifier_sum_undersampling.csv")


target_0 = result_undersampling[result_undersampling['isTopPerformer'] == False]
target_1 = result_undersampling[result_undersampling['isTopPerformer'] == True]
print(len(target_0), len(target_1))
result_undersampling.head(15)

0 2991


,isExpelled,courseCompleted,isFailed,weighted_score,isTopPerformer_value,isTopPerformer,studentId
0,0,0,0,-0.166191,1,True,11450
1,0,0,0,-0.166191,1,True,7855
2,0,0,0,-0.166191,1,True,12753
3,0,0,0,-0.166191,1,True,14717
4,0,0,0,-0.166191,1,True,7455
5,0,0,0,-0.166191,1,True,6958
6,0,0,0,-0.166191,1,True,8004
7,0,0,0,-0.166191,1,True,8575
8,0,0,0,-0.166191,1,True,15668
9,0,0,0,2.060376,1,True,9155


In [147]:
my_submission = pd.read_csv('RandomForestClassifier_sum_undersampling.csv')
print(my_submission.columns)
print(len(my_submission))
my_submission.head()

Index(['studentId', 'isTopPerformer'], dtype='object')
2991


,studentId,isTopPerformer
0,11450,True
1,7855,True
2,12753,True
3,14717,True
4,7455,True


In [ ]:
# other features